In [9]:
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import os

In [17]:
# Carregar variáveis de ambiente
load_dotenv()
api_q_key = os.getenv('Q_API_KEY_NIAMH')

# Criar cliente OpenAI com a chave da API correta
if not api_q_key:
    print("Erro: Chave da API não encontrada. Verifique se a variável de ambiente está configurada corretamente.")
else:
    print("Chave da API encontrada.")
    print(f"Chave: {api_q_key[:10]}...")  # Exibe apenas parte da chave para segurança

    try:
        # Criar cliente OpenAI
        openai = OpenAI(api_key=api_q_key)

        # Teste de conexão com a API
        response = openai.chat.completions.create(
            model="gpt-4o",  # Substituir por um modelo válido
            messages=[{"role": "user", "content": "Teste de conexão"}],
            max_tokens=10
        )
        print("Conexão com a API realizada com sucesso.")
        print("Resposta:", response.choices[0].message.content)
    except Exception as e:
        print("Erro ao conectar com a API:")
        print(str(e))  # Exibir detalhes do erro


Chave da API encontrada.
Chave: sk-proj-9n...
Conexão com a API realizada com sucesso.
Resposta: Parece que sua conexão está funcionando! Como posso


In [21]:
# Carregar variáveis de ambiente
load_dotenv()
api_q_key = os.getenv('Q_API_KEY_NIAMH')
openai = OpenAI(api_key=api_q_key)

# Carregar a planilha original
file_path = "backup.csv"  # Substituir pelo caminho correto do arquivo
df = pd.read_csv(file_path)

# Remover duplicatas para garantir que cada combinação seja única
unique_combinations = df.drop_duplicates(subset=["area_de_conhecimento", "disciplina", "assunto_pai"])

# Função para gerar questão e resposta ideal
def generate_question_and_answer(area, discipline, topic):
    prompt = (
        f"Crie uma questão discursiva de nível de concurso público brasileiro "
        f"sobre o tema '{topic}', pertencente à disciplina '{discipline}', "
        f"dentro da área de conhecimento '{area}'. "
        f"A questão deve exigir uma resposta elaborada sendo apenas texto. "
        f"Em seguida, forneça a resposta ideal para essa questão em formato apenas de texto sem html ou markdown."
    )

    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "Você é um especialista em criação de questões para concursos públicos no Brasil."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )
    return response.choices[0].message.content

# Gerar questões e respostas
generated_data = []

for _, row in unique_combinations.iterrows():
    area = row["area_de_conhecimento"]
    discipline = row["disciplina"]
    topic = row["assunto_pai"]

    for attempt in range(3):  # Tenta no máximo 3 vezes
        try:
            qa_text = generate_question_and_answer(area, discipline, topic)
            print(f"Resposta gerada para '{topic}': {qa_text}\n")

            if "\n\n" in qa_text:
                question, answer = qa_text.split("\n\n", 1)
            else:
                question, answer = qa_text, "Erro: Resposta não está no formato esperado"
            break  # Se conseguiu gerar, sai do loop
        except Exception as e:
            print(f"Erro ao gerar para '{topic}', tentativa {attempt + 1}: {e}")
            question, answer = "Erro ao gerar questão", str(e)
            time.sleep(2)  # Espera 2 segundos antes de tentar novamente

    generated_data.append([area, discipline, topic, question, answer])

# Criar DataFrame com os resultados
df_results = pd.DataFrame(
    generated_data,
    columns=["area_de_conhecimento", "disciplina", "assunto_pai", "questao", "resposta"]
)

# Salvar em um arquivo CSV garantindo compatibilidade de caracteres especiais
output_file_path = "questoes_discursivas.csv"
df_results.to_csv(output_file_path, index=False, encoding="utf-8-sig")

print(f"Arquivo salvo em: {output_file_path}")

Resposta gerada para 'Ciclo Orçamentário': **Questão:**

Explique detalhadamente as fases do ciclo orçamentário no Brasil, destacando a importância de cada etapa no processo de elaboração, aprovação, execução e controle do orçamento público. Em sua resposta, discorra sobre os principais instrumentos utilizados em cada fase, os atores envolvidos e os prazos estabelecidos pela legislação vigente. Além disso, analise como essas etapas contribuem para a eficiência e transparência na gestão dos recursos públicos.

**Resposta Ideal:**

O ciclo orçamentário no Brasil é composto por quatro fases principais: elaboração, aprovação, execução e controle/avaliação do orçamento. Cada uma dessas fases desempenha um papel crucial na gestão eficiente e transparente dos recursos públicos.

A primeira fase, a elaboração, é de responsabilidade do Poder Executivo. Nessa etapa, o governo formula a proposta orçamentária, que deve ser compatível com o Plano Plurianual (PPA) e a Lei de Diretrizes Orçamentárias